In [1]:
!git clone https://github.com/rezabayati1/background_remove.git

Cloning into 'background_remove'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 70 (delta 8), reused 58 (delta 2), pack-reused 0
Unpacking objects: 100% (70/70), 7.35 MiB | 3.14 MiB/s, done.


In [2]:
%cd background_remove/

/content/background_remove


In [ ]:
!pip install ./transparent-background/

In [ ]:
!pip install ./image-upscale/

In [ ]:
!pip install ./faceorienter

In [ ]:
!pip install face_recognition

In [12]:
import cv2
from PIL import Image
from transparent_background import Remover
import torch
from RealESRGAN import RealESRGAN
from PIL import ImageDraw
import dlib
import numpy as np
from background_remove.faceorienter.faceorienter import FaceOrienter
import face_recognition

class ImageProcessor:
    def __init__(self):
        self.remover = Remover(ckpt='transparent-background/latest.pth', device="cuda:0")
        self.device = torch.device('cuda')
        self.model = RealESRGAN(self.device, scale=4)
        self.model.load_weights('transparent-background/weights/RealESRGAN_x4.pth', download=True)

    def remove_background(self, image_path):
        img = Image.open(image_path).convert('RGB')
        out = self.remover.process(img, type='white')
        output_path = 'bg_removed_sample.jpg'
        Image.fromarray(out).save(output_path)
        return output_path

    def upscale_image(self, image_path):
        path_to_image = image_path
        image = Image.open(path_to_image)
        sr_image = self.model.predict(image)
        output_path = 'upscaled_sample.jpg'
        sr_image.save(output_path)
        return output_path

    def fix_orientation(self, image_path):
        fo = FaceOrienter(image_path)
        orientation = fo.predict_orientation()
        fixed_image_path = 'fixed_orientation_sample.jpg'
        fo.fix_orientation(fixed_image_path)
        return fixed_image_path

    def rotate_and_resize_face(self, image_path):
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image)
        face_landmarks = face_recognition.face_landmarks(image_array)

        if len(face_landmarks) == 0:
            print("No face detected in the image.")
            return None

        face_landmark = face_landmarks[0]
        left_eye = face_landmark['left_eye']
        right_eye = face_landmark['right_eye']
        dx = right_eye[0][0] - left_eye[3][0]
        dy = right_eye[0][1] - left_eye[3][1]
        angle = np.degrees(np.arctan2(dy, dx))
        rotated_image = image.rotate(angle, expand=True, fillcolor='white')
        new_image = Image.new('RGB', image.size, (255, 255, 255))
        paste_position = (
            (new_image.width - rotated_image.width) // 2,
            (new_image.height - rotated_image.height) // 2
        )
        new_image.paste(rotated_image, paste_position)
        resized_image = new_image.resize((400, 600))
        resized_image.save('result.jpg')
        return 'result.jpg'


# Example usage
processor = ImageProcessor()
image_path = 'sample.jpg'

# Remove background
output_path = processor.remove_background(image_path)

# Upscale image
output_path = processor.upscale_image(output_path)

# Fix orientation
output_path = processor.fix_orientation(output_path)

# Rotate and resize face
output_path = processor.rotate_and_resize_face(output_path)


Settings -> Mode=base, Device=cuda:0, Torchscript=disabled
